In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
rental_data = pd.read_csv('data_for_regression2.csv')

In [4]:
rental_data

,beds_baths_interaction,beds,baths,longitude,latitude,beds_to_baths_ratio,street_number,month_available,daysOnMarket,PhotosNum,...,Median Home Value,hasVideo,occupied_to_vacant_ratio,propertyType_rental,propertyType_condo,crime_rate,White Alone_ratio,schools_in_precinct,price_bin,price
0,1.0,1,1.0,-73.922293,40.662186,0.500000,171.0,11,1.0,4,...,565200.0,1,13.157442,1,0,1.893937,0.066448,41,0,1855.0
1,2.0,2,1.0,-73.924122,40.663817,1.000000,93.0,11,4.0,5,...,565200.0,1,13.157442,1,0,1.893937,0.066448,41,0,2235.0
2,1.0,1,1.0,-73.917055,40.668506,0.500000,501.0,11,4.0,7,...,565200.0,1,13.157442,1,0,2.036421,0.066448,56,0,1750.0
3,1.0,1,1.0,-73.917750,40.657349,0.500000,1115.0,11,6.0,4,...,565200.0,1,13.157442,1,0,1.893937,0.066448,41,0,1850.0
4,3.0,3,1.0,-73.922602,40.659825,1.500000,1075.0,11,7.0,9,...,565200.0,1,13.157442,1,0,1.893937,0.066448,41,0,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6966,4.0,4,1.0,-73.889854,40.823948,2.000000,1027.0,11,121.0,0,...,570600.0,0,25.148305,0,0,1.346101,0.107863,33,1,4070.0
6967,9.0,3,3.0,-74.249470,40.513058,0.750000,80.0,11,6.0,18,...,785200.0,0,20.458824,0,0,0.615876,0.907082,15,1,3800.0
6968,2.0,2,1.0,-74.009330,40.709580,1.000000,11.0,11,5.0,5,...,886400.0,0,5.254689,0,0,1.258965,0.462066,25,1,3750.0
6969,1.0,1,1.0,-74.006035,40.709370,0.500000,88.0,11,10.0,0,...,886400.0,0,5.254689,0,0,1.258965,0.462066,25,1,3200.0


In [35]:
# Load the saved model
with open('best_model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [9]:
bins = [0, 3000, 6000, float('inf')]  # Low, Medium, High
labels = [0, 1, 2]  # Numerical labels for the new bins

In [25]:
X = rental_data.drop(['price', 'price_bin'], axis=1)
y = rental_data['price']

# Split data for regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
# Create regression models for each bin
regression_models = {}
for bin_label in labels:
    bin_data = rental_data[rental_data['price_bin'] == bin_label]
    X_bin = bin_data.drop(['price', 'price_bin'], axis=1)
    y_bin = bin_data['price']
    
    # Split data for regression
    X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin, y_bin, test_size=0.2, random_state=42)
    
    # Gradient Boosting Regression
    reg_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    reg_model.fit(X_train_bin, y_train_bin)

    # Evaluate the model
    y_pred_bin = reg_model.predict(X_test_bin)
    mse = mean_squared_error(y_test_bin, y_pred_bin)
    rmse = mean_squared_error(y_test_bin, y_pred_bin, squared=False)
#     print(f"MSE for Gradient Boosting in {bin_label} bin: {mse}")
    print(f"RMSE for Gradient Boosting in {bin_label} bin: {rmse}")
    regression_models[bin_label] = reg_model


RMSE for Gradient Boosting in 0 bin: 277.8504145219084
RMSE for Gradient Boosting in 1 bin: 526.4091499879554
RMSE for Gradient Boosting in 2 bin: 9416.401466191135


In [19]:
# Assuming df is your DataFrame with 'price' and 'price_bin' columns
average_price_by_bin = rental_data.groupby('price_bin')['price'].mean()
# median_price_by_bin = rental_data.groupby('price_bin')['price'].median()
# Print the result
print(average_price_by_bin)
# print(median_price_by_bin)

price_bin
0     2472.276428
1     3978.895371
2    14369.701342
Name: price, dtype: float64


In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Define hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 200, 300],          # Number of trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],     # Learning rate
    'max_depth': [3, 5, 7, 10],                  # Maximum depth of trees
    'min_samples_split': [2, 5, 10],             # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],               # Minimum samples in a leaf
    'subsample': [0.7, 0.8, 0.9, 1.0]            # Fraction of samples for training
}

# Create regression models for each bin
regression_models = {}
for bin_label in labels:
    bin_data = rental_data[rental_data['price_bin'] == bin_label]
    X_bin = bin_data.drop(['price', 'price_bin'], axis=1)
    y_bin = bin_data['price']
    
    # Split data for regression
    X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin, y_bin, test_size=0.2, random_state=42)
    
    # Initialize Gradient Boosting Regressor
    gbr = GradientBoostingRegressor(random_state=42)
    
    # Randomized Search for Hyperparameter Tuning
    random_search = RandomizedSearchCV(
        estimator=gbr,
        param_distributions=param_dist,
        n_iter=50,  # Number of parameter combinations to try
        cv=3,       # 3-fold cross-validation
        scoring='neg_root_mean_squared_error',  # Optimization metric
        random_state=42,
        verbose=2,
        n_jobs=-1
    )
    
    # Fit the RandomizedSearchCV
    random_search.fit(X_train_bin, y_train_bin)
    
    # Best model for the bin
    best_model = random_search.best_estimator_
    regression_models[bin_label] = best_model
    
    # Evaluate the best model
    y_pred_bin = best_model.predict(X_test_bin)
    mse = mean_squared_error(y_test_bin, y_pred_bin)
    rmse = mean_squared_error(y_test_bin, y_pred_bin, squared=False)
    print(f"Best RMSE for {bin_label} bin: {rmse}")
    print(f"Best Parameters for {bin_label} bin: {random_search.best_params_}")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best RMSE for 0 bin: 274.7358427369589
Best Parameters for 0 bin: {'subsample': 0.7, 'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.05}
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best RMSE for 1 bin: 526.8296227581312
Best Parameters for 1 bin: {'subsample': 0.7, 'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.05}
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best RMSE for 2 bin: 9422.774729102375
Best Parameters for 2 bin: {'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.1}
[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=300, subsample=0.7; total time=   3.0s
[CV] END learning_rate=0.1, max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=300, su

[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=10, n_estimators=300, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=10, n_estimators=300, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=7, min_samples_leaf=4, min_samples_split=5, n_estimators=100, subsample=0.9; total time=   0.9s
[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.7; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=50, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=2, min_samples_split=5, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.1, max_depth=10, min_samples

[CV] END learning_rate=0.1, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=300, subsample=0.9; total time=   1.9s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=10, n_estimators=300, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.7; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=50, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=2, min_samples_split=5, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.05, max_depth=7, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   0.5s
[CV] END learning_rate=0.1, max_depth=5, min_samples_lea

[CV] END learning_rate=0.1, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=300, subsample=0.9; total time=   1.9s
[CV] END learning_rate=0.1, max_depth=5, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.9; total time=   0.6s
[CV] END learning_rate=0.1, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.4s
[CV] END learning_rate=0.1, max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=300, subsample=0.8; total time=   1.0s
[CV] END learning_rate=0.1, max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=3, min_samples_leaf=4, min_samples_split=5, n_estimators=300, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.1, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=300, subsample=0.9; total time=   3.8s
[CV] END learning_rate=0.1, max_depth=10, min_samples_leaf

[CV] END learning_rate=0.01, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=300, subsample=0.7; total time=   2.9s
[CV] END learning_rate=0.2, max_depth=7, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.7; total time=   0.4s
[CV] END learning_rate=0.1, max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=100, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=3, min_samples_leaf=4, min_samples_split=5, n_estimators=300, subsample=0.7; total time=   1.1s
[CV] END learning_rate=0.05, max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=300, subsample=1.0; total time=   4.1s
[CV] END learning_rate=0.1, max_depth=7, min_samples_leaf=1, min_samples_split=5, n_estimators=200, subsample=1.0; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200, subsample=0.8; total time=   2.2s
[CV] END learning_rate=0.1, max_depth=10, min_samples_l

# bin 0

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Data for Bin 0
bin_data_0 = rental_data[rental_data['price_bin'] == 0]
X_bin_0 = bin_data_0.drop(['price', 'price_bin'], axis=1)
y_bin_0 = bin_data_0['price']

# Split data for regression
X_train_bin_0, X_test_bin_0, y_train_bin_0, y_test_bin_0 = train_test_split(X_bin_0, y_bin_0, test_size=0.2, random_state=42)

# Define refined parameter grid
param_dist_0 = {
    'n_estimators': [250, 300, 350, 400],
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'subsample': [0.6, 0.7, 0.8, 1.0]
}

# Initialize Gradient Boosting Regressor
gbr_0 = GradientBoostingRegressor(random_state=42)

# Randomized Search
random_search_0 = RandomizedSearchCV(
    estimator=gbr_0,
    param_distributions=param_dist_0,
    n_iter=30,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Fit RandomizedSearchCV
random_search_0.fit(X_train_bin_0, y_train_bin_0)

# Evaluate the best model
y_pred_bin_0 = random_search_0.best_estimator_.predict(X_test_bin_0)
rmse_0 = mean_squared_error(y_test_bin_0, y_pred_bin_0, squared=False)
print(f"Refined RMSE for Bin 0: {rmse_0}")
print(f"Refined Best Parameters for Bin 0: {random_search_0.best_params_}")


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Refined RMSE for Bin 0: 275.5017537009487
Refined Best Parameters for Bin 0: {'subsample': 0.8, 'n_estimators': 400, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_depth': 3, 'learning_rate': 0.03}


# bin 1

In [22]:
# Data for Bin 1
bin_data_1 = rental_data[rental_data['price_bin'] == 1]
X_bin_1 = bin_data_1.drop(['price', 'price_bin'], axis=1)
y_bin_1 = bin_data_1['price']

# Split data for regression
X_train_bin_1, X_test_bin_1, y_train_bin_1, y_test_bin_1 = train_test_split(X_bin_1, y_bin_1, test_size=0.2, random_state=42)

# Define refined parameter grid
param_dist_1 = {
    'n_estimators': [250, 300, 350, 400],
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'subsample': [0.6, 0.7, 0.8, 1.0]
}

# Initialize Gradient Boosting Regressor
gbr_1 = GradientBoostingRegressor(random_state=42)

# Randomized Search
random_search_1 = RandomizedSearchCV(
    estimator=gbr_1,
    param_distributions=param_dist_1,
    n_iter=30,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Fit RandomizedSearchCV
random_search_1.fit(X_train_bin_1, y_train_bin_1)

# Evaluate the best model
y_pred_bin_1 = random_search_1.best_estimator_.predict(X_test_bin_1)
rmse_1 = mean_squared_error(y_test_bin_1, y_pred_bin_1, squared=False)
print(f"Refined RMSE for Bin 1: {rmse_1}")
print(f"Refined Best Parameters for Bin 1: {random_search_1.best_params_}")


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Refined RMSE for Bin 1: 525.3842442057336
Refined Best Parameters for Bin 1: {'subsample': 0.8, 'n_estimators': 350, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_depth': 3, 'learning_rate': 0.05}


# bin 2

In [23]:
# Data for Bin 2
bin_data_2 = rental_data[rental_data['price_bin'] == 2]
X_bin_2 = bin_data_2.drop(['price', 'price_bin'], axis=1)
y_bin_2 = bin_data_2['price']

# Split data for regression
X_train_bin_2, X_test_bin_2, y_train_bin_2, y_test_bin_2 = train_test_split(X_bin_2, y_bin_2, test_size=0.2, random_state=42)

# Define refined parameter grid
param_dist_2 = {
    'n_estimators': [250, 300, 350, 400],
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'subsample': [0.6, 0.7, 0.8, 1.0]
}

# Initialize Gradient Boosting Regressor
gbr_2 = GradientBoostingRegressor(random_state=42)

# Randomized Search
random_search_2 = RandomizedSearchCV(
    estimator=gbr_2,
    param_distributions=param_dist_2,
    n_iter=30,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Fit RandomizedSearchCV
random_search_2.fit(X_train_bin_2, y_train_bin_2)

# Evaluate the best model
y_pred_bin_2 = random_search_2.best_estimator_.predict(X_test_bin_2)
rmse_2 = mean_squared_error(y_test_bin_2, y_pred_bin_2, squared=False)
print(f"Refined RMSE for Bin 2: {rmse_2}")
print(f"Refined Best Parameters for Bin 2: {random_search_2.best_params_}")


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Refined RMSE for Bin 2: 9111.47242582495
Refined Best Parameters for Bin 2: {'subsample': 1.0, 'n_estimators': 250, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 2, 'learning_rate': 0.07}


In [24]:
regressors = [gbr_0,gbr_1,gbr_2]

In [26]:
def predict_price(data_point, classifier, regressors, bins):
    # Step 1: Predict the bin
    predicted_bin = classifier.predict([data_point])[0]
    
    # Step 2: Use the corresponding regression model
    regressor = regressors[predicted_bin]
    predicted_price = regressor.predict([data_point])[0]
    
    return predicted_bin, predicted_price

[CV] END learning_rate=0.05, max_depth=3, min_samples_leaf=3, min_samples_split=5, n_estimators=300, subsample=1.0; total time=   1.4s
[CV] END learning_rate=0.07, max_depth=2, min_samples_leaf=4, min_samples_split=5, n_estimators=250, subsample=1.0; total time=   0.9s
[CV] END learning_rate=0.05, max_depth=4, min_samples_leaf=5, min_samples_split=5, n_estimators=400, subsample=0.6; total time=   1.7s
[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=3, min_samples_split=5, n_estimators=300, subsample=0.6; total time=   1.2s
[CV] END learning_rate=0.03, max_depth=4, min_samples_leaf=3, min_samples_split=4, n_estimators=350, subsample=0.8; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=3, min_samples_leaf=4, min_samples_split=5, n_estimators=250, subsample=1.0; total time=   1.1s
[CV] END learning_rate=0.1, max_depth=2, min_samples_leaf=3, min_samples_split=6, n_estimators=400, subsample=0.6; total time=   0.8s
[CV] END learning_rate=0.03, max_depth=3, min_samples_lea

[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=4, min_samples_split=4, n_estimators=250, subsample=0.7; total time=   1.2s
[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=5, min_samples_split=4, n_estimators=400, subsample=1.0; total time=   2.5s
[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=3, min_samples_split=5, n_estimators=300, subsample=0.6; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=2, min_samples_leaf=4, min_samples_split=4, n_estimators=400, subsample=0.8; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=2, min_samples_leaf=5, min_samples_split=4, n_estimators=250, subsample=0.7; total time=   0.8s
[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=3, min_samples_split=6, n_estimators=300, subsample=0.8; total time=   1.5s
[CV] END learning_rate=0.05, max_depth=4, min_samples_leaf=3, min_samples_split=5, n_estimators=250, subsample=0.6; total time=   0.9s
[CV] END learning_rate=0.05, max_depth=4, min_samples_l

[CV] END learning_rate=0.07, max_depth=4, min_samples_leaf=4, min_samples_split=4, n_estimators=250, subsample=0.7; total time=   1.2s
[CV] END learning_rate=0.07, max_depth=2, min_samples_leaf=4, min_samples_split=5, n_estimators=250, subsample=1.0; total time=   0.8s
[CV] END learning_rate=0.03, max_depth=2, min_samples_leaf=3, min_samples_split=5, n_estimators=300, subsample=1.0; total time=   1.0s
[CV] END learning_rate=0.05, max_depth=4, min_samples_leaf=3, min_samples_split=6, n_estimators=400, subsample=0.6; total time=   1.7s
[CV] END learning_rate=0.03, max_depth=4, min_samples_leaf=5, min_samples_split=5, n_estimators=400, subsample=0.7; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=4, min_samples_leaf=4, min_samples_split=5, n_estimators=250, subsample=0.7; total time=   1.2s
[CV] END learning_rate=0.05, max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=300, subsample=0.6; total time=   0.6s
[CV] END learning_rate=0.05, max_depth=4, min_samples_le

In [39]:
# Example usage
data_point = X_test.iloc[0]  # Example data point
predicted_bin, predicted_price = predict_price(data_point, clf, regression_models, labels)
print(f"Predicted Bin: {predicted_bin}, Predicted Price: {predicted_price}, Real Price = {y_test.iloc[0]}")

Predicted Bin: 0, Predicted Price: 2644.9680488086874, Real Price = 2600.0


/Users/nikhilsoni/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/nikhilsoni/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
